# SPECTRE MODEL TEST

In [9]:
from scapy.all import rdpcap
import numpy as np
import tensorflow as tf
from datetime import datetime
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow_io as tfio
from collections import defaultdict
import tensorflow as tf
from keras.models import load_model

In [ ]:
# KAFKA IMPORT

In [10]:
malicious_data = np.load('/home/aryn/spectre-dev/dataset/Malware_Capture_Facility_Project/PCAP-NPY/2015-09-10_winlinux.npy', allow_pickle=True)

df = pd.DataFrame(malicious_data)
with pd.option_context('display.max_rows', 5, 'display.max_columns', 5):
    print(df)

                0           1   ...            98            99
0         1.000000   72.000000  ...  0.000000e+00  0.000000e+00
1         1.000000   72.000000  ...  0.000000e+00  0.000000e+00
...            ...         ...  ...           ...           ...
539687  631.603849  949.999817  ...  3.835983e-25  6.462192e-16
539688  632.498958  949.999817  ...  3.212164e-25  3.826360e-16

[539689 rows x 100 columns]


# PREDICTION

In [11]:
# Load TF Saved_model
model = tf.saved_model.load("/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/SavedModel")

2023-05-07 17:25:14.460877: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-07 17:25:14.473327: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
# load the model from an HDF5 file
model_h5 = load_model('/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/spectre_ddos_h5.h5')

In [12]:
#predictions = model(features_array)
result = model(malicious_data)

2023-05-07 17:25:42.463601: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 431751200 exceeds 10% of free system memory.


ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * <tf.Tensor 'inputs:0' shape=(539689, 100) dtype=float64>
    * False
    * None
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='input_1')
    * True
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='input_1')
    * False
    * None
  Keyword arguments: {}

Option 3:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='inputs')
    * True
    * None
  Keyword arguments: {}

Option 4:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='inputs')
    * False
    * None
  Keyword arguments: {}

In [ ]:
#predictions = model(features_array)
result = model_h5(malicious_data)